# Arize <> LangChain Integration

**Let's get started on using Langchain with Arize!** ✨



Arize is an LLM Observability Platform for real-time monitoring and analysis of your LLM applications.

Use Arize and LangChain together to effectively monitor the performance of your LLM agents, identify areas that require improvement, and make prompt engineering and troubleshooting decisions about your LLM applications. With Arize and LangChain together, data scientists and machine learning engineers can ensure that their LLM applications are running at peak efficiency, enabling them to deliver improved results and drive greater value for their organizations.

# Step 0. Install Dependencies, Import Libraries, Use GPU 📚

Import LangChain, Arize, and Arize CallBack Handler for integration between two tools.

⚠️ Use a GPU to save time generating embeddings. Click on 'Runtime', select 'Change Runtime Type' and
select 'GPU'.

In [ ]:
!pip3 install -q langchain
!pip3 install -q arize
!pip3 install -q 'arize[AutoEmbeddings]'
!pip3 install -q openai

In [2]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.callbacks.manager import AsyncCallbackManager
from langchain.callbacks import ArizeCallbackHandler
from langchain.llms import OpenAI

# Step 1. Import and Setup Arize Client

The first step is to setup our Arize client. After that we will log the data from LangChain driven application into Arize.

Retrieve your Arize `API_KEY` and `SPACE_KEY` from your Space Settings page, and paste them in the set-up section below.

We will also set up some metadata and the `ArizeCallBackHandler` to use while logging.




<img src="https://storage.googleapis.com/arize-assets/fixtures/copy-keys.png" width="700">

In [14]:
SPACE_KEY = "YOUR_SPACE_KEY"
API_KEY =  "YOUR_API_KEY"

if SPACE_KEY == "YOUR_SPACE_KEY" or API_KEY == "YOUR_API_KEY":
    raise ValueError("❌ CHANGE SPACE AND API KEYS")

# Define callback handler for Arize
arize_callback = ArizeCallbackHandler(
model_id="llm-langchain-demo",
model_version="1.0",
SPACE_KEY=SPACE_KEY,
API_KEY=API_KEY
)

  arize.utils.logging | WARNING | No available GPU has been detected. The use of GPU acceleration is strongly recommended. You can check for GPU availability by running `torch.cuda.is_available()`
  arize.utils.logging | INFO | Downloading pre-trained model 'distilbert-base-uncased'
  arize.utils.logging | INFO | Downloading tokenizer for 'distilbert-base-uncased'
✅ Arize client setup done! Now you can start using Arize!


In [ ]:
#Put your OPENAI API Key here!
%env OPENAI_API_KEY="YOUR OPEN API KEY"

## Step 2: Define LLM with ArizeCallBack Handler
Use the callback handler we defined above within the LLM with LangChain.

In [16]:
manager = AsyncCallbackManager([StdOutCallbackHandler(), arize_callback])
llm = OpenAI(temperature=0, callback_manager=manager, verbose=True)

## Step 3: Test LLM Responses and Logging into Arize
Use some simple prompts to test if the LLM works properly and each prompt-response pair is logged into Arize with embeddings.

In [17]:
llm_result = llm.generate(["Tell me an interesting fact about pandas.","Explain the concept of overfitting in 2 sentences."])

  arize.utils.logging | INFO | Generating embedding vectors


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

  arize.utils.logging | INFO | Generating embedding vectors


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

  arize.utils.logging | WARNING | prediction_label_column_name was not provided, a default prediction label equal to 1 will be set for GENERATIVE_LLM models.
  arize.utils.logging | WARNING | actual_label_column_name was not provided. Some metrics that require actual labels, e.g. correctness or accuracy, may not be computed.
  arize.utils.logging | INFO | Success! Check out your data at https://app.arize.com/organizations/QWNjb3VudE9yZ2FuaXphdGlvbjoxOTI1/spaces/U3BhY2U6MjAzNg==/models/modelName/llm-langchain-demo?selectedTab=dataIngestion
✅ Successfully logged data to Arize!
  arize.utils.logging | INFO | Generating embedding vectors


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

  arize.utils.logging | INFO | Generating embedding vectors


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

  arize.utils.logging | WARNING | prediction_label_column_name was not provided, a default prediction label equal to 1 will be set for GENERATIVE_LLM models.
  arize.utils.logging | WARNING | actual_label_column_name was not provided. Some metrics that require actual labels, e.g. correctness or accuracy, may not be computed.
  arize.utils.logging | INFO | Success! Check out your data at https://app.arize.com/organizations/QWNjb3VudE9yZ2FuaXphdGlvbjoxOTI1/spaces/U3BhY2U6MjAzNg==/models/modelName/llm-langchain-demo?selectedTab=dataIngestion
✅ Successfully logged data to Arize!


## Step 4: Test LLM Chain and Agents with Arize Integration

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, callback_manager=manager)

template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.
Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, callback_manager=manager)

overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], verbose=True, callback_manager=manager
)

test_prompts = [
    {
        "input": "documentary about pandas who are about be extinct because of global warming"
    },
    {"input": "once upon a time in hollywood"},
    {"input": "the best mo observability tooling"},
]
overall_chain.apply(test_prompts)